In [1]:
import nltk
nltk.download('punkt')#Sentence tokenizer

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

# Preprocessing

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('E:\intents.json').read() # read json file
intents = json.loads(data_file) # load json file

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)# add each elements into list
        #combination between patterns and intents
        documents.append((w, intent['tag']))#add single element into end of list
        # add to tag in our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
nltk.download('wordnet') #lexical database for the English language

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [7]:
nltk.download('omw-1.4')

[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [8]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")
# classes = intents[tag]
print (len(classes), "classes\n", classes, "\n")
# words = all words, vocabulary
print (len(words), "unique lemmatized words\n", words, "\n")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

474 documents
 [(['Hi'], 'greeting'), (['How', 'are', 'you', '?'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyy'], 'greeting'), (['whatsup'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Bye'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creators'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creators'], 'creator'), (['who', 'created', 'you'], 'creator')

# Training Model

In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words
    pattern_words = doc[0]
    # convert pattern_words in lower case
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    print(pattern_words)
    # create bag of words array,if word match found in current pattern then put 1 otherwise 0.[row * colm(263)]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # in output array 0 value for each tag ang 1 value for matched tag.[row * colm(8)]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle training and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test. X - patterns(words), Y - intents(tags)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

In [10]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [11]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
print("First layer:",model.layers[0].get_weights()[0])

First layer: [[ 0.08408255 -0.02685943 -0.07847324 ...  0.00138114  0.06345785
   0.03685518]
 [ 0.06636757 -0.11714464  0.06884535 ...  0.10562807  0.07087097
   0.07177339]
 [-0.09774764 -0.06931043  0.03624517 ...  0.07844757  0.00144821
   0.10379452]
 ...
 [-0.02285793  0.04095415  0.00724418 ... -0.1010677  -0.07604878
   0.03601716]
 [-0.00230931  0.01374532  0.04544864 ...  0.03586281  0.0015587
  -0.01023498]
 [-0.04639385  0.0564672  -0.05328331 ... -0.07524565  0.02057923
   0.01484402]]


In [12]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
95/95 [==============================] - 4s 7ms/step - loss: 3.7776 - accuracy: 0.0527
Epoch 2/200
95/95 [==============================] - 1s 6ms/step - loss: 3.6060 - accuracy: 0.1245: 0s - loss: 3.6464 - ac
Epoch 3/200
95/95 [==============================] - 1s 6ms/step - loss: 3.3984 - accuracy: 0.1688
Epoch 4/200
95/95 [==============================] - 1s 6ms/step - loss: 3.0695 - accuracy: 0.2511
Epoch 5/200
95/95 [==============================] - 1s 6ms/step - loss: 2.8015 - accuracy: 0.2932
Epoch 6/200
95/95 [==============================] - 1s 7ms/step - loss: 2.4869 - accuracy: 0.3586
Epoch 7/200
95/95 [==============================] - 1s 6ms/step - loss: 2.2215 - accuracy: 0.4241
Epoch 8/200
95/95 [==============================] - 1s 6ms/step - loss: 2.0529 - accuracy: 0.4726
Epoch 9/200
95/95 [==============================] - 1s 7ms/step - loss: 1.8243 - accuracy: 0.5464
Epoch 10/200
95/95 [==============================] - 1s 7ms/step - loss: 1.6338 - ac

95/95 [==============================] - 1s 8ms/step - loss: 0.1383 - accuracy: 0.9494
Epoch 83/200
95/95 [==============================] - 1s 6ms/step - loss: 0.1889 - accuracy: 0.9430
Epoch 84/200
95/95 [==============================] - 1s 6ms/step - loss: 0.1855 - accuracy: 0.9241
Epoch 85/200
95/95 [==============================] - 1s 5ms/step - loss: 0.1645 - accuracy: 0.9388
Epoch 86/200
95/95 [==============================] - 1s 7ms/step - loss: 0.1683 - accuracy: 0.9430
Epoch 87/200
95/95 [==============================] - 1s 6ms/step - loss: 0.1544 - accuracy: 0.9494
Epoch 88/200
95/95 [==============================] - 1s 6ms/step - loss: 0.1519 - accuracy: 0.9536
Epoch 89/200
95/95 [==============================] - 1s 5ms/step - loss: 0.1215 - accuracy: 0.9641
Epoch 90/200
95/95 [==============================] - 1s 6ms/step - loss: 0.1667 - accuracy: 0.9388
Epoch 91/200
95/95 [==============================] - 1s 6ms/step - loss: 0.1269 - accuracy: 0.9599
Epoch 92/200


95/95 [==============================] - 1s 6ms/step - loss: 0.0961 - accuracy: 0.9620
Epoch 163/200
95/95 [==============================] - 1s 6ms/step - loss: 0.1149 - accuracy: 0.9557
Epoch 164/200
95/95 [==============================] - 1s 6ms/step - loss: 0.0847 - accuracy: 0.9684
Epoch 165/200
95/95 [==============================] - 1s 6ms/step - loss: 0.0922 - accuracy: 0.9641
Epoch 166/200
95/95 [==============================] - 1s 6ms/step - loss: 0.1137 - accuracy: 0.9641
Epoch 167/200
95/95 [==============================] - 1s 6ms/step - loss: 0.1386 - accuracy: 0.9536
Epoch 168/200
95/95 [==============================] - 1s 6ms/step - loss: 0.0874 - accuracy: 0.9662: 0s - loss: 0.1210 
Epoch 169/200
95/95 [==============================] - 1s 7ms/step - loss: 0.1099 - accuracy: 0.9578: 1s - loss: 0.0
Epoch 170/200
95/95 [==============================] - 1s 6ms/step - loss: 0.1027 - accuracy: 0.9599
Epoch 171/200
95/95 [==============================] - 1s 6ms/step - 

In [14]:
from keras.models import load_model
model = load_model('chatbot_model.h5')

In [15]:
# Utility Methods
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    #print(sentence_words)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    #print(sentence_words)
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
#     print(sentence_words)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
#     print(bag)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                
                if show_details:
                    print ("found in bag: %s" % w)
                #print ("found in bag: %s" % w)
#     print(bag)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
#     print(p)
    res = model.predict(np.array([p]))[0]
#     print(res)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
#     print(results)
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
#     print(results)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
#     print(return_list)
    return return_list

In [16]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
#     print(tag)
    list_of_intents = intents_json['intents']
#     print(list_of_intents)
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
#     print(ints)
    res = getResponse(ints, intents)
#     print(res)
    return res

In [ ]:
start = True
while start:
    query = input('Enter Message:')
    if query in ['quit','exit','bye']:
        start = False
        continue
    try:
        res = chatbot_response(query)
        print(res)
    except:
        print('You may need to rephrase your question.')